1. Introdução
Este notebook detalha o processo de análise de dados de acidentes de trânsito em Porto Alegre, a criação de um agente inteligente para recomendação de medidas de moderação de tráfego e a construção de um modelo preditivo para estimar a ocorrência de acidentes futuros

1.1. Configuração do Ambiente e Importação de Bibliotecas

In [3]:
# Manipulação e Análise de Dados
import pandas as pd
import numpy as np

# Visualização de Dados
import matplotlib.pyplot as plt
import seaborn as sns

# Geocodificação e APIs
import openmeteo_requests
import requests_cache
from retry_requests import retry




2. Coleta, Limpeza e Enriquecimento dos Dados

2.1. Carga e Limpeza Inicial do Dataset 

In [2]:
# Carregar o dataset
df = pd.read_csv('acidentes.csv', sep=';', low_memory=False)

# Limpeza inicial dos nomes das colunas
df.columns = df.columns.str.strip().str.lower()

# Converter colunas de data e hora
df['data'] = pd.to_datetime(df['data'], errors='coerce')
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S.%f', errors='coerce').dt.time

# Combinar data e hora em uma única coluna datetime
df['data_hora'] = df.apply(lambda r: pd.datetime.combine(r['data'], r['hora']) if pd.notnull(r['data']) and pd.notnull(r['hora']) else pd.NaT, axis=1)
df = df.dropna(subset=['data_hora'])

# Tratar coordenadas inválidas (0 ou nulas)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df.loc[df['latitude'] == 0, 'latitude'] = np.nan
df.loc[df['longitude'] == 0, 'longitude'] = np.nan

# Criar colunas agregadas para vítimas e veículos
df['total_vitimas'] = df['feridos'] + df['fatais']
veiculo_cols = ['auto', 'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'patinete']
df['total_veiculos'] = df[veiculo_cols].sum(axis=1)

# Criar coluna de severidade
def define_severidade(row):
    if row['fatais'] > 0:
        return 'Fatal'
    elif row['feridos'] > 0:
        return 'Com Feridos'
    else:
        return 'Sem Vítimas'
df['severidade'] = df.apply(define_severidade, axis=1)

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'acidentes.csv'

2.2. Enriquecimento Geoespacial via Geocodificação
Muitos registros não possuem coordenadas de latitude e longitude. Usaremos a API Nominatim (OpenStreetMap) para geocodificar os endereços (logradouros e cruzamentos) e preencher os dados faltantes.

Aviso: A execução desta célula pode demorar várias horas devido às limitações da API gratuita do Nominatim. Para uma execução mais rápida, o código pode ser adaptado para rodar em um subconjunto dos dados ou utilizar uma API de geocodificação paga."

In [4]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": -30.12,
	"longitude": -51.23,
	"start_date": "2022-01-01",
	"end_date": "2022-12-31",
	"hourly": "precipitation",
	"timezone": "America/Sao_Paulo",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_precipitation = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: -30.125°N -51.25°E
Elevation: 46.0 m asl
Timezone: b'America/Sao_Paulo'b'GMT-3'
Timezone difference to GMT+0: -10800s

Hourly data
                           date  precipitation
0    2022-01-01 03:00:00+00:00            0.0
1    2022-01-01 04:00:00+00:00            0.0
2    2022-01-01 05:00:00+00:00            0.0
3    2022-01-01 06:00:00+00:00            0.0
4    2022-01-01 07:00:00+00:00            0.0
...                        ...            ...
8755 2022-12-31 22:00:00+00:00            0.0
8756 2022-12-31 23:00:00+00:00            0.0
8757 2023-01-01 00:00:00+00:00            0.0
8758 2023-01-01 01:00:00+00:00            0.0
8759 2023-01-01 02:00:00+00:00            0.0

[8760 rows x 2 columns]
